In [1]:
!pip install sklearn boto3 botocore prefect==0.14.6 --user

In [2]:
from prefect import task, Flow, Parameter, Client
from prefect.run_configs import KubernetesRun
from prefect.schedules import IntervalSchedule
from prefect.storage import S3, Local

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

from datetime import timedelta

import numpy as np
import pandas as pd

import mlflow
import requests

In [3]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

@task
def fetch_data():
    csv_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    data = pd.read_csv(csv_url, sep=";")
    return data
 
@task
def train_model(data, mlflow_experiment_id, alpha=0.5, l1_ratio=0.5):
    mlflow.set_tracking_uri("http://mlflow.mlflow:5000")
 
    train, test = train_test_split(data)
 
    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    
    with mlflow.start_run(experiment_id=mlflow_experiment_id):
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
 
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
 
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
 
        mlflow.sklearn.log_model(lr, "model")

In [4]:
import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio.minio:9000'
os.environ['MLFLOW_TRACKING_URI'] = 'http://mlflow.mlflow:5000' # need for MLFlow model API
os.environ['ARTIFACT_ROOT'] = 's3://mlflow'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admintest'

In [5]:
domain = "mlops.ritza-route53.com"
username = "gareth@ritza.co" 
password = "DkguP5GsB9yiPk8"
s3_bucket = "another-mlops-bucket"

auth_url = f"https://{domain}/.ory/kratos/public/self-service/login/api"
prefect_url = f"http://10.107.111.92:4200/graphql"

prefect_project_name = "wine-quality-project"
docker_image = "drtools/prefect:wine-classifier-3"


def create_prefect_flow():
    run_config = KubernetesRun(
        labels=["dev"],
        service_account_name="prefect-server-serviceaccount",
        image=docker_image
    )
    storage = Local()
    storage = S3(bucket="mlflow", client_options=dict(endpoint_url=os.getenv("MLFLOW_S3_ENDPOINT_URL"), 
                                                     aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
                                                     aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")))

    #session_token = get_prefect_token()
    prefect_client = Client(api_server=prefect_url)
    print(prefect_client.graphql(
    {
        'query': {
            'flow': {
                'id'
            }
        }
    }
    ))
    print(prefect_client._active_tenant_id)
    print(prefect_client._api_token)
    schedule = IntervalSchedule(interval=timedelta(minutes=1))

    with Flow("train-wine-quality-model-v8", schedule=schedule, storage=storage, run_config=run_config) as flow:
        data = fetch_data()
        train_model(data=data, mlflow_experiment_id=1, alpha=0.3, l1_ratio=0.3)

    #flow.run()
    #prefect_client.create_project(project_name=prefect_project_name)
    training_flow_id = prefect_client.register(flow, project_name=prefect_project_name)
    
    flow_run_id = prefect_client.create_flow_run(flow_id=training_flow_id, run_name=f"run {prefect_project_name}")
    
create_prefect_flow()


{'data': {'flow': [{'id': 'a6c57087-1c51-489b-b1d1-0df6dfcfaff8'}, {'id': 'ec1638a3-dfdf-4d0e-9ba6-926bbc1d8407'}, {'id': 'efabcacb-67a2-4285-a835-325e1a5b0d1d'}, {'id': 'fc350a6b-3ea3-4af1-ae8c-e7fd304cc007'}, {'id': '78720017-b02e-4bfe-9e29-c33401d96be4'}, {'id': '8640028f-20e0-49a8-8660-a1d1475b019d'}, {'id': '0e28e0e7-7e15-4778-8048-005cf1bfd81c'}, {'id': '2ad3760c-bed1-4157-9569-b77a5998bf80'}, {'id': '4cea0d90-63a3-427e-82d5-bead11ac0ab2'}, {'id': '6dafd82d-79c1-4449-9b66-aa008806efd2'}, {'id': '556190d6-d6d7-4695-8dab-d239d682b374'}, {'id': 'bc2ff5b4-5ef6-483f-9ee7-bc3036b00373'}, {'id': '68783cde-9225-4ff2-95d5-c8e3fd24186c'}, {'id': '4db0ce2d-ab93-4cf0-995c-4ba87bacba4c'}, {'id': '6316d87c-d3d0-4175-bcb7-a779e6d11d36'}, {'id': '8fad4acb-2231-4ba4-99b0-e2899a7e4c55'}, {'id': 'ad213eb4-8ca6-49aa-8e9d-6affa0f74bf3'}, {'id': '28472935-0109-45b7-b1a3-1582688b6358'}, {'id': '7b753971-5e44-4600-908c-3af5938b556a'}, {'id': 'd344baa9-1e5e-4b51-9c3b-193484fada8a'}, {'id': 'ce5cf10b-9a9b

<ipython-input-5-89bcb38c1463>:45: UserWarning: No result handler was specified on your Flow. Cloud features such as input caching and resuming task runs from failure may not work properly.
  training_flow_id = prefect_client.register(flow, project_name=prefect_project_name)


ClientError: 400 Client Error: Bad Request for url: http://10.107.111.92:4200/graphql

The following error messages were provided by the GraphQL server:

    GRAPHQL_VALIDATION_FAILED: Cannot query field "user" on type "Query".

The GraphQL query was:

    query {
            user {
                default_membership {
                    tenant {
                        slug
                }
            }
        }
    }

The passed variables were:

    null


In [7]:
import prefect 

prefect.__version__

'0.14.12'